# NV-Retriever 방법론 실험

1. Teacher - Base Model 선정
    - Teacher 후보 모델 (한국어 / 영어)
        - https://huggingface.co/DeepMount00/Llama-3.1-8b-ITA
        - https://huggingface.co/AIDX-ktds/ktdsbaseLM-v0.12-based-on-openchat3.5
    - Base 후보 모델 (한국어 / 영어)
        - https://huggingface.co/intfloat/multilingual-e5-large

2. Dataset 구성
    - 사용할 데이터셋 리스트업 (한국어 / 영어)
        - 한국어
            - (QA) nayohan/Sujet-Finance-Instruct-177k-ko
            - (QA) BCCard/BCCard-Finance-Kor-QnA
            - (corpus) https://huggingface.co/datasets/amphora/korfin-asc?row=3
        - 영어
            - (QA) FinLang/investopedia-instruction-tuning-dataset
            - (sentiment) Fingpt/fingpt-sentiment-train
    - pair 구성하기
        1. positive pair : QA set의 경우 Query - Answer
            - Answer 없는 경우 : Title - Passage, **BM25** 등
        2. Negative pair : in-batch + Hard negative
            - Teacher Model을 통해 Hard negative Mining 진행
            - Query당 최소 1개 ~ 최대 4개의 Hard negative (Batch / Base Model 크기에 따라 달라짐)
        
        **+) 금융 도메인 / 데이터 특성에 따라 pair 선정 방식 별도 추가**
        
3. Instruction Tuning
    - prefix 형식 {task_prefix} : {query} → 데이터셋에 따라 변경
    - Tuning 진행 (LoRA 적용)
    - A6000x4 + deepspeed stage 3 가 최소 사양
    
4. Evaluation
    1. (한국어) Eval dataset 구축 (KorFinMTEB)
    2. (영어) 현존하는 FinMTEB 평가

## 1. Teacher - base model

## 1. 한국어 데이터셋
### 1-1. BCcard QA
- https://huggingface.co/datasets/BCCard/BCCard-Finance-Kor-QnA
- 수집한 데이터셋

In [ ]:
import pandas as pd

bccard_df = pd.read_json("hf://datasets/BCCard/BCCard-Finance-Kor-QnA/bccard-finance-qna.jsonl", lines=True)
bccard_df.head()

In [ ]:
bccard_df.shape

### 데이터 전처리


In [ ]:
bccard_df.reset_index(drop=True, inplace=True)
bccard_df = bccard_df.rename(columns={"instruction" : "Query", "output" : "Answer"})
bccard_df.head()

### BM25Okapi

- 원본 논문 (NV-Retriever) 에는 BM25 성능이 좋지 않았으나, 금융 문장은 키워드나 단어가 중요한 만큼 TF-IDF 기반의 해당 방법론에 대해서도 실험을 진행

In [4]:
from konlpy.tag import Okt

# 한국어는 형태소 쪼개기 이후 분석하는게 좋다고 함 - 영어가 섞여있으므로 그냥 쪼개기도 해보기
# M1에서 konlpy 실행 X 이슈로.. 일단은 그냥 split
def tokenizing(sent):
    okt = Okt()
    return okt.morphs(sent)


### BM25 (All)

In [8]:
import pandas as pd
from rank_bm25 import BM25Okapi
from tqdm import tqdm
import numpy as np

# 데이터 준비
def prepare_data(data):
    queries = data['Query'].tolist()
    answers = data['Answer'].tolist()
    return queries, answers

# BM25 모델 초기화
def initialize_bm25(corpus):
    tokenized_corpus = [doc.split() for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25

# BM25 점수 정규화 함수
def normalize_scores(scores):
    scores = np.array(scores)
    min_score = scores.min()
    max_score = scores.max()
    if max_score - min_score == 0:  # 모든 점수가 동일한 경우
        return np.zeros_like(scores)  # 정규화 점수를 0으로 설정
    return (scores - min_score) / (max_score - min_score)

# Hard Negative Mining
def mine_hard_negatives(data, bm25, max_neg=4):
    results = []
    for index, row in tqdm(data.iterrows(), total=len(data)):
        query = row['Query']
        positive_answer = row['Answer']

        # Query의 토큰화
        tokenized_query = tokenizing(query)

        # BM25 점수 계산 및 정규화
        scores = bm25.get_scores(tokenized_query)
        normalized_scores = normalize_scores(scores)

        # Positive 유사도 계산
        pos_score = normalized_scores[index]

        # max_neg_score_threshold 계산
        max_neg_score_threshold = pos_score * 0.95

        # Hard Negative 후보 필터링
        negative_candidates = [(i, normalized_scores[i]) for i in range(len(scores)) if \
                               normalized_scores[i] <= max_neg_score_threshold and i != index]
                               
        # 유사도가 높은 순으로 정렬
        negative_candidates = sorted(negative_candidates, key=lambda x: x[1], reverse=True)

        # 최대 max_neg개의 Hard Negative 선택
        hard_negatives = negative_candidates[:max_neg]
        
        # Hard Negative 추가
        for neg in hard_negatives:
            results.append({
                'Query': query,
                'Positive Answer': positive_answer,
                'Hard Negative': data.iloc[neg[0]]['Answer'],
                'Positive Score' : pos_score,
                'Negative Score': neg[1]
            })

    return pd.DataFrame(results)

In [ ]:
# # sample

# bccard_df_sample = pd.read_csv("./data/bccard_df_sample.csv")

# # 데이터 준비
# queries, answers = prepare_data(bccard_df_sample)

# # BM25 모델 초기화
# bm25 = initialize_bm25(answers)

# # Hard Negative Mining 실행
# mined_data = mine_hard_negatives(bccard_df_sample, bm25)


In [ ]:
# 전체
# 데이터 준비
queries, answers = prepare_data(bccard_df)

# BM25 모델 초기화
bm25 = initialize_bm25(answers)

# Hard Negative Mining 실행
mined_data = mine_hard_negatives(bccard_df, bm25)


In [ ]:
mined_data.head()

In [8]:
# bccard_df.to_csv("./data/bccard_df.csv", index=False)
mined_data.to_csv("./data/bm25.csv", index=False)

### 실험 수정 - minmax top_k = 100

In [4]:
import pandas as pd
from rank_bm25 import BM25Okapi
from tqdm import tqdm
import numpy as np

# 데이터 준비
def prepare_data(data):
    queries = data['Query'].tolist()
    answers = data['Answer'].tolist()
    return queries, answers

# BM25 모델 초기화
def initialize_bm25(corpus):
    tokenized_corpus = [doc.split() for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25

# BM25 점수 0~1 Scaling 함수
def normalize_scores(scores):
    scores = np.array(scores)
    min_score = scores.min()
    max_score = scores.max()
    if max_score - min_score == 0:  # 모든 점수가 동일한 경우
        return np.zeros_like(scores)  # 정규화 점수를 0으로 설정
    return (scores - min_score) / (max_score - min_score)

# Hard Negative Mining
def mine_hard_negatives(data, bm25, top_k=100, max_neg=4):
    results = []
    for index, row in tqdm(data.iterrows(), total=len(data)):
        query = row['Query']
        positive_answer = row['Answer']

        # Query의 토큰화
        tokenized_query = query.split()  # 간단한 토큰화

        # BM25 점수 계산
        scores = bm25.get_scores(tokenized_query)

        # 상위 100개의 점수 선택
        top_100_indices = np.argsort(scores)[-top_k:][::-1]
        top_100_scores = scores[top_100_indices]

        # 0~1 Scaling 적용
        normalized_scores = normalize_scores(top_100_scores)

        # Positive 유사도 계산 (정규화된 점수에서 위치 기반으로 접근)
        if index in top_100_indices:
            pos_index = np.where(top_100_indices == index)[0][0]
            pos_score = normalized_scores[pos_index]
        else:
            pos_score = 0  # Positive가 top 100에 없는 경우

        # max_neg_score_threshold 계산
        max_neg_score_threshold = pos_score * 0.95

        # Hard Negative 후보 필터링
        negative_candidates = [(i, normalized_scores[j]) for j, i in enumerate(top_100_indices) if \
                               normalized_scores[j] <= max_neg_score_threshold and i != index]

        # 유사도가 높은 순으로 정렬
        negative_candidates = sorted(negative_candidates, key=lambda x: x[1], reverse=True)

        # 최대 max_neg개의 Hard Negative 선택
        hard_negatives = negative_candidates[:max_neg]

        # Hard Negative 추가
        for neg in hard_negatives:
            results.append({
                'Query': query,
                'Positive Answer': positive_answer,
                'Hard Negative': data.iloc[neg[0]]['Answer'],
                'Positive Score': pos_score,
                'Negative Score': neg[1]
            })

    return pd.DataFrame(results)


In [5]:
# sample
bccard_df_sample = pd.read_csv("./data/bccard_df_sample.csv")

# 데이터 준비
queries, answers = prepare_data(bccard_df_sample)

# BM25 모델 초기화
bm25 = initialize_bm25(answers)

# Hard Negative Mining 실행
mined_data = mine_hard_negatives(bccard_df_sample, bm25, 100)

100%|██████████| 2841/2841 [00:08<00:00, 344.09it/s]


In [6]:
mined_data

,Query,Positive Answer,Hard Negative,Positive Score,Negative Score
0,LTI 규제가 변동될 때 기존 대출의 조건도 바뀌나요?,LTI 규제가 변동되더라도 기존에 받은 대출의 조건은 바뀌지 않습니다. 대출은 대출...,"LTI 규제가 없는 상황에서는 대출 한도는 대출자의 신용 등급, 소득, 기존 부채 ...",1.00000,0.521175
1,LTI 규제가 변동될 때 기존 대출의 조건도 바뀌나요?,LTI 규제가 변동되더라도 기존에 받은 대출의 조건은 바뀌지 않습니다. 대출은 대출...,대출 규제가 강화되면 금융기관은 대출 금리를 인상할 가능성이 있습니다. 이는 대출의...,1.00000,0.514133
2,LTI 규제가 변동될 때 기존 대출의 조건도 바뀌나요?,LTI 규제가 변동되더라도 기존에 받은 대출의 조건은 바뀌지 않습니다. 대출은 대출...,"금리 인하 요구권을 통해 금리가 인하된 후에도, 시장 금리 변화나 대출 조건에 따라...",1.00000,0.378502
3,LTI 규제가 변동될 때 기존 대출의 조건도 바뀌나요?,LTI 규제가 변동되더라도 기존에 받은 대출의 조건은 바뀌지 않습니다. 대출은 대출...,"LTI는 주로 신용대출에 적용되지만, 주택담보대출에는 보통 LTV(Loan to V...",1.00000,0.371950
4,공매도와 관련된 불법 행위는 어떤 것들이 있나요?,"공매도와 관련된 불법 행위로는 무차입 공매도, 미공개 정보 이용, 시세 조종 등이 ...","한국에서는 자동차 튜닝 시 안전기준과 환경기준을 준수해야 하며, 불법 튜닝은 처벌 ...",1.00000,0.509408
...,...,...,...,...,...
11038,기준금리와 재정 정책은 어떻게 연관되나요?,기준금리와 재정 정책은 경제 안정과 성장을 위한 두 가지 주요 정책 도구입니다. 기...,포워드 가이던스와 재정 정책이 협력할 때 경제 안정성에 중요한 이점을 제공합니다. ...,1.00000,0.308168
11039,"현금서비스 이용 시, 120만원을 인출하고, 이자율이 23%일 때 50일 동안 사용...",이자는 \( 120만원 \times \frac{23\%}{365} \times 50...,"30대들은 주로 신용카드와 체크카드를 통해 결제합니다. 특히, 신용카드의 경우 무이...",0.04362,0.040819
11040,"현금서비스 이용 시, 120만원을 인출하고, 이자율이 23%일 때 50일 동안 사용...",이자는 \( 120만원 \times \frac{23\%}{365} \times 50...,"네, 로스트아크 카드를 해외에서 사용할 경우 수수료가 부과될 수 있습니다. 자세한 ...",0.04362,0.040819
11041,"현금서비스 이용 시, 120만원을 인출하고, 이자율이 23%일 때 50일 동안 사용...",이자는 \( 120만원 \times \frac{23\%}{365} \times 50...,트레이딩 봇은 자동으로 거래를 실행하는 프로그램입니다. 사용자가 설정한 매개변수에 ...,0.04362,0.040296


## 1-2. Dataset - Naver Finnews

In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np

naver_news = pd.read_csv("./data/naver_main_news_2024.csv")
naver_news.head()

In [ ]:
naver_news.shape

In [3]:
naver_news.dropna(subset=['cleaned_text'], inplace=True)

In [ ]:
naver_news.shape

- title - passage를 positive pair로

In [ ]:
import ast

naver_news['cleaned_text'] = naver_news['cleaned_text'].apply(lambda x : ' '.join(ast.literal_eval(x)))
naver_news.head()

In [6]:
news_pseudo_qa = pd.DataFrame()
news_pseudo_qa['Query'] = naver_news['title']
news_pseudo_qa['Answer'] = naver_news['cleaned_text']

news_pseudo_qa.reset_index(drop=True, inplace=True)

In [ ]:
news_pseudo_qa

- frac = 0.1

In [ ]:
news_sample = news_pseudo_qa.sample(frac=0.1).reset_index(drop=True)
news_sample

In [25]:
news_sample.to_csv("./data/news_sample.csv", index=False)

In [ ]:
# 전체
# 데이터 준비
queries, answers = prepare_data(news_sample)

# BM25 모델 초기화
bm25 = initialize_bm25(answers)

# Hard Negative Mining 실행
mined_data = mine_hard_negatives(news_sample, bm25)

mined_data.to_csv("./data/NaverNews_bm25_sample_hard_negative.csv", index=False)

In [ ]:
# 전체
# 데이터 준비
queries, answers = prepare_data(news_pseudo_qa)

# BM25 모델 초기화
bm25 = initialize_bm25(answers)

# Hard Negative Mining 실행
mined_data = mine_hard_negatives(news_pseudo_qa, bm25)

mined_data.to_csv("./data/NaverNews_bm25_hard_negative.csv", index=False)

## BM25를 다른 방식으로 시도?

- top-k 100에서 softmax 취하기 -> 전체에서 할 경우 너무 극단적임